In [51]:
import pandas as pd

In [83]:
#import one of datasets

users_df1 = pd.read_csv("../data/users_books_dataset/BX-Users.csv",sep=';') 
ratings_df1 = pd.read_csv("../data/users_books_dataset/BX-Book-Ratings.csv",sep=';') 
books_df1 = pd.read_csv("../data/users_books_dataset/BX-Books.csv",sep=';') 

In [92]:
books_df1

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271375,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271377,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [146]:
books_df1[books_df1['ISBN'] == '0439023483']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L


In [236]:
#import dataset goodbooks

book_tags_goods = pd.read_csv("../data/good_books/book_tags.csv")
books_goods = pd.read_csv("../data/good_books/books.csv") 
ratings_goods = pd.read_csv("../data/good_books/ratings.csv") 
tags_goods = pd.read_csv("../data/good_books/tags.csv") 
to_read_goods = pd.read_csv("../data/good_books/to_read.csv") 

In [237]:
books_goods[books_goods['isbn13'] == 9.780439e+12]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url


In [238]:
books_goods

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...


In [239]:
import decimal

for i in range(len(books_goods)):
    print(i)
    print(f"es {books_goods.iloc[i]['isbn13']}")
    print(f"es {decimal.Decimal(books_goods.iloc[i]['isbn13'])}")
    value =  decimal.Decimal(books_goods.iloc[i]['isbn13'])
    books_goods.loc[i,books_goods.iloc[i]['isbn13']] = value
    print(f"se transforma {value}")

print('-------- OK ---------')
    

0
es 9780439023480.0
es 9780439023480
se transforma 9780439023480
1
es 9780439554930.0
es 9780439554930
se transforma 9780439554930
2
es 9780316015840.0
es 9780316015840
se transforma 9780316015840
3
es 9780061120080.0
es 9780061120080
se transforma 9780061120080
4
es 9780743273560.0
es 9780743273560
se transforma 9780743273560
5
es 9780525478810.0
es 9780525478810
se transforma 9780525478810
6
es 9780618260300.0
es 9780618260300
se transforma 9780618260300
7
es 9780316769170.0
es 9780316769170
se transforma 9780316769170
8
es 9781416524790.0
es 9781416524790
se transforma 9781416524790
9
es 9780679783270.0
es 9780679783270
se transforma 9780679783270
10
es 9781594480000.0
es 9781594480000
se transforma 9781594480000
11
es 9780062024040.0
es 9780062024040
se transforma 9780062024040
12
es 9780451524940.0
es 9780451524940
se transforma 9780451524940
13
es 9780452284240.0
es 9780452284240
se transforma 9780452284240
14
es 9780553296980.0
es 9780553296980
se transforma 9780553296980
15
es

/Users/gabrielcuevavaldez/miniconda3/envs/recommender/lib/python3.9/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


se transforma 9780340839940
126
es 9780316346630.0
es 9780316346630
se transforma 9780316346630
127
es 9781451648540.0
es 9781451648540
se transforma 9781451648540
128
es 9780451163970.0
es 9780451163970
se transforma 9780451163970
129
es 9780684830490.0
es 9780684830490
se transforma 9780684830490
130
es 9780142000660.0
es 9780142000660
se transforma 9780142000660
131
es 9781401308580.0
es 9781401308580
se transforma 9781401308580
132
es 9780451528830.0
es 9780451528830
se transforma 9780451528830
133
es 9781416914300.0
es 9781416914300
se transforma 9781416914300
134
es 9780553573430.0
es 9780553573430
se transforma 9780553573430
135
es 9780060759960.0
es 9780060759960
se transforma 9780060759960
136
es 9780440242950.0
es 9780440242950
se transforma 9780440242950
137
es 9780142437260.0
es 9780142437260
se transforma 9780142437260
138
es 9781594744760.0
es 9781594744760
se transforma 9781594744760
139
es 9780307270000.0
es 9780307270000
se transforma 9780307270000
140
es 9780804139020

se transforma 9780451205770
274
es 9780525476890.0
es 9780525476890
se transforma 9780525476890
275
es 9780553803720.0
es 9780553803720
se transforma 9780553803720
276
es 9780062255660.0
es 9780062255660
se transforma 9780062255660
277
es 9780060513060.0
es 9780060513060
se transforma 9780060513060
278
es 9780751565360.0
es 9780751565360
se transforma 9780751565360
279
es 9780061726840.0
es 9780061726840
se transforma 9780061726840
280
es 9780385339700.0
es 9780385339700
se transforma 9780385339700
281
es 9780061139380.0
es 9780061139380
se transforma 9780061139380
282
es 9780060853980.0
es 9780060853980
se transforma 9780060853980
283
es 9780140042600.0
es 9780140042600
se transforma 9780140042600
284
es 9781476729080.0
es 9781476729080
se transforma 9781476729080
285
es 9780446579930.0
es 9780446579930
se transforma 9780446579930
286
es 9780451191150.0
es 9780451191150
se transforma 9780451191150
287
es 9780425172900.0
es 9780425172900
se transforma 9780425172900
288
es 9780380395870

se transforma 9780553212280
402
es 9781590302260.0
es 9781590302260
se transforma 9781590302260
403
es 9780385344230.0
es 9780385344230
se transforma 9780385344230
404
es 9780385334200.0
es 9780385334200
se transforma 9780385334200
405
es 9780874774240.0
es 9780874774240
se transforma 9780874774240
406
es 9780060838720.0
es 9780060838720
se transforma 9780060838720
407
es 9780739467350.0
es 9780739467350
se transforma 9780739467350
408
es 9780375508420.0
es 9780375508420
se transforma 9780375508420
409
es 9780385474540.0
es 9780385474540
se transforma 9780385474540
410
es 9780385338690.0
es 9780385338690
se transforma 9780385338690
411
es 9781881273160.0
es 9781881273160
se transforma 9781881273160
412
es nan
es NaN
se transforma NaN
413
es 9781416589650.0
es 9781416589650
se transforma 9781416589650
414
es 9780312532760.0
es 9780312532760
se transforma 9780312532760
415
es 9780590032490.0
es 9780590032490
se transforma 9780590032490
416
es 9780743298020.0
es 9780743298020
se transform

...

es 9780345419580.0
es 9780345419580
se transforma 9780345419580
9433
es 9780380803260.0
es 9780380803260
se transforma 9780380803260
9434
es 9780670913500.0
es 9780670913500
se transforma 9780670913500
9435
es 9780307393840.0
es 9780307393840
se transforma 9780307393840
9436
es 9780393346770.0
es 9780393346770
se transforma 9780393346770
9437
es 9781905294430.0
es 9781905294430
se transforma 9781905294430
9438
es 9780316013310.0
es 9780316013310
se transforma 9780316013310
9439
es 9780451210460.0
es 9780451210460
se transforma 9780451210460
9440
es 9780785122460.0
es 9780785122460
se transforma 9780785122460
9441
es 9780060959520.0
es 9780060959520
se transforma 9780060959520
9442
es 9781573451990.0
es 9781573451990
se transforma 9781573451990
9443
es 9788756795460.0
es 9788756795460
se transforma 9788756795460
9444
es nan
es NaN
se transforma NaN
9445
es 9780007345630.0
es 9780007345630
se transforma 9780007345630
9446
es 9781857232520.0
es 9781857232520
se transforma 9781857232520
94

es nan
es NaN
se transforma NaN
9558
es 9780743525880.0
es 9780743525880
se transforma 9780743525880
9559
es 9780441017090.0
es 9780441017090
se transforma 9780441017090
9560
es 9780688104640.0
es 9780688104640
se transforma 9780688104640
9561
es 9780446618760.0
es 9780446618760
se transforma 9780446618760
9562
es 9780942299790.0
es 9780942299790
se transforma 9780942299790
9563
es 9780786836610.0
es 9780786836610
se transforma 9780786836610
9564
es 9780316261140.0
es 9780316261140
se transforma 9780316261140
9565
es 9781417642170.0
es 9781417642170
se transforma 9781417642170
9566
es 9780374316420.0
es 9780374316420
se transforma 9780374316420
9567
es 9780743413080.0
es 9780743413080
se transforma 9780743413080
9568
es 9780062498530.0
es 9780062498530
se transforma 9780062498530
9569
es 9789992195840.0
es 9789992195840
se transforma 9789992195840
9570
es 9780345511560.0
es 9780345511560
se transforma 9780345511560
9571
es 9780988474410.0
es 9780988474410
se transforma 9780988474410
95

se transforma 9780804140840
9680
es 9781462003920.0
es 9781462003920
se transforma 9781462003920
9681
es 9781416935710.0
es 9781416935710
se transforma 9781416935710
9682
es 9789992195760.0
es 9789992195760
se transforma 9789992195760
9683
es 9780399156040.0
es 9780399156040
se transforma 9780399156040
9684
es nan
es NaN
se transforma NaN
9685
es 9780340796150.0
es 9780340796150
se transforma 9780340796150
9686
es 9780330519470.0
es 9780330519470
se transforma 9780330519470
9687
es 9781590304500.0
es 9781590304500
se transforma 9781590304500
9688
es 9780375701240.0
es 9780375701240
se transforma 9780375701240
9689
es 9783822859830.0
es 9783822859830
se transforma 9783822859830
9690
es 9780385482390.0
es 9780385482390
se transforma 9780385482390
9691
es 9780451232690.0
es 9780451232690
se transforma 9780451232690
9692
es nan
es NaN
se transforma NaN
9693
es 9780451225090.0
es 9780451225090
se transforma 9780451225090
9694
es 9781421515880.0
es 9781421515880
se transforma 9781421515880
9

se transforma 9780230711260
9804
es 9780671027580.0
es 9780671027580
se transforma 9780671027580
9805
es 9781421534540.0
es 9781421534540
se transforma 9781421534540
9806
es 9781518783880.0
es 9781518783880
se transforma 9781518783880
9807
es 9781401301380.0
es 9781401301380
se transforma 9781401301380
9808
es 9781607749720.0
es 9781607749720
se transforma 9781607749720
9809
es 9780399153940.0
es 9780399153940
se transforma 9780399153940
9810
es 9780062101310.0
es 9780062101310
se transforma 9780062101310
9811
es 9780099465640.0
es 9780099465640
se transforma 9780099465640
9812
es 9781571316520.0
es 9781571316520
se transforma 9781571316520
9813
es 9780553298020.0
es 9780553298020
se transforma 9780553298020
9814
es 9780307460450.0
es 9780307460450
se transforma 9780307460450
9815
es nan
es NaN
se transforma NaN
9816
es 9780061229700.0
es 9780061229700
se transforma 9780061229700
9817
es 9780717283540.0
es 9780717283540
se transforma 9780717283540
9818
es 9781402179620.0
es 97814021796

es 9780310328180.0
es 9780310328180
se transforma 9780310328180
9932
es 9780140565330.0
es 9780140565330
se transforma 9780140565330
9933
es 9780967089740.0
es 9780967089740
se transforma 9780967089740
9934
es 9780425204310.0
es 9780425204310
se transforma 9780425204310
9935
es 9780061148480.0
es 9780061148480
se transforma 9780061148480
9936
es 9781596435710.0
es 9781596435710
se transforma 9781596435710
9937
es 9780061779760.0
es 9780061779760
se transforma 9780061779760
9938
es 9781594487710.0
es 9781594487710
se transforma 9781594487710
9939
es 9780316012390.0
es 9780316012390
se transforma 9780316012390
9940
es 9788807813020.0
es 9788807813020
se transforma 9788807813020
9941
es 9781442486590.0
es 9781442486590
se transforma 9781442486590
9942
es 9780866114260.0
es 9780866114260
se transforma 9780866114260
9943
es 9780670026560.0
es 9780670026560
se transforma 9780670026560
9944
es nan
es NaN
se transforma NaN
9945
es 9780385413720.0
es 9780385413720
se transforma 9780385413720
99

In [240]:
books_goods.iloc[1]['isbn13'])

SyntaxError: unmatched ')' (3073739275.py, line 1)

In [235]:
books_goods

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#import dataset readbooks

books_reads = pd.read_csv("../data/read_books/books.csv")


In [175]:
books_reads['isbn13'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
11122    False
11123    False
11124    False
11125    False
11126    False
Name: isbn13, Length: 11127, dtype: bool

In [233]:
books_reads[books_reads['isbn13'] == 9780439785969]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.


In [77]:
books_reads.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [176]:
for i in range(len(books_reads)):
    print(books_reads.loc[i]['isbn13'])
    print('-------- new ---------')

9780439785969
-------- new ---------
9780439358071
-------- new ---------
9780439554893
-------- new ---------
9780439655484
-------- new ---------
9780439682589
-------- new ---------
9780976540601
-------- new ---------
9780439827607
-------- new ---------
9780517226957
-------- new ---------
9780345453747
-------- new ---------
9781400052929
-------- new ---------
9780739322208
-------- new ---------
9780517149256
-------- new ---------
9780767908184
-------- new ---------
9780767915069
-------- new ---------
9780767910439
-------- new ---------
9780767903868
-------- new ---------
9780767903820
-------- new ---------
9780060920081
-------- new ---------
9780380713806
-------- new ---------
9780380727506
-------- new ---------
9780380715435
-------- new ---------
9780345538376
-------- new ---------
9780618517657
-------- new ---------
9780618346257
-------- new ---------
9780618260584
-------- new ---------
9780618391004
-------- new ---------
9780618510825
-------- new ---------
9

9780226307947
-------- new ---------
9780195135923
-------- new ---------
9780140443332
-------- new ---------
9780872203907
-------- new ---------
9780140441123
-------- new ---------
9780374527051
-------- new ---------
9780812216271
-------- new ---------
9780226307640
-------- new ---------
9780521539814
-------- new ---------
9781400041923
-------- new ---------
9780812216714
-------- new ---------
9780486292427
-------- new ---------
9781419150012
-------- new ---------
9780226307862
-------- new ---------
9780452011670
-------- new ---------
9780451527844
-------- new ---------
9780156027649
-------- new ---------
9780140444254
-------- new ---------
9780872205857
-------- new ---------
9780679431329
-------- new ---------
9780140440287
-------- new ---------
9780192835888
-------- new ---------
9781580495936
-------- new ---------
9780451621603
-------- new ---------
9780521617352
-------- new ---------
9781416500339
-------- new ---------
9780452007178
-------- new ---------
9

9780890093894
-------- new ---------
9780001000391
-------- new ---------
9780394448046
-------- new ---------
9780143036388
-------- new ---------
9780141182674
-------- new ---------
9780375711169
-------- new ---------
9783829602266
-------- new ---------
9780679725404
-------- new ---------
9780143034544
-------- new ---------
9780807744680
-------- new ---------
9780195137897
-------- new ---------
9780071460828
-------- new ---------
9780316346627
-------- new ---------
9780786887170
-------- new ---------
9780060521998
-------- new ---------
9780141439563
-------- new ---------
9780393960693
-------- new ---------
9780664241582
-------- new ---------
9780060935467
-------- new ---------
9780060888695
-------- new ---------
9780061205699
-------- new ---------
9780761501664
-------- new ---------
9780486287591
-------- new ---------
9781416500391
-------- new ---------
9781556434952
-------- new ---------
9780192834423
-------- new ---------
9781931082303
-------- new ---------
9

9780553380163
-------- new ---------
9780767923224
-------- new ---------
9780895260475
-------- new ---------
9780802715524
-------- new ---------
9780517223123
-------- new ---------
9780743564410
-------- new ---------
9780764596865
-------- new ---------
9780609602195
-------- new ---------
9780792272663
-------- new ---------
9780374271077
-------- new ---------
9780701127664
-------- new ---------
9788420466552
-------- new ---------
9780670037568
-------- new ---------
9788466617352
-------- new ---------
9780747236047
-------- new ---------
9780785118954
-------- new ---------
9783453215283
-------- new ---------
9782266111577
-------- new ---------
9782221089569
-------- new ---------
9780380817931
-------- new ---------
9780380817924
-------- new ---------
9780316017442
-------- new ---------
9780060506049
-------- new ---------
9780312989484
-------- new ---------
9780553572940
-------- new ---------
9780380817160
-------- new ---------
9780312282998
-------- new ---------
9

-------- new ---------
9780618711666
-------- new ---------
9780452270831
-------- new ---------
9781932416640
-------- new ---------
9780375725784
-------- new ---------
9781400033546
-------- new ---------
9781400095568
-------- new ---------
9781932416138
-------- new ---------
9780439635776
-------- new ---------
9780809166589
-------- new ---------
9780142001431
-------- new ---------
9780553804768
-------- new ---------
9781416516934
-------- new ---------
9780812977363
-------- new ---------
9780385333849
-------- new ---------
9780791071687
-------- new ---------
9780743422000
-------- new ---------
9780385333900
-------- new ---------
9780425134061
-------- new ---------
9780312353766
-------- new ---------
9780827230286
-------- new ---------
9780826415745
-------- new ---------
9782070417940
-------- new ---------
9780674023857
-------- new ---------
9780786918140
-------- new ---------
9780670886951
-------- new ---------
9788466313599
-------- new ---------
9781400076192
-

...

9780613371650
-------- new ---------
9780439629393
-------- new ---------
9780439933384
-------- new ---------
9780439933414
-------- new ---------
9780618477944
-------- new ---------
9780696211522
-------- new ---------
9780393058475
-------- new ---------
9780199205646
-------- new ---------
9783442545537
-------- new ---------
9781400032051
-------- new ---------
9780898706833
-------- new ---------
9780385515504
-------- new ---------
9780307275561
-------- new ---------
9780446580281
-------- new ---------
9780765354952
-------- new ---------
9780446611237
-------- new ---------
9780446612760
-------- new ---------
9780515094657
-------- new ---------
9781412086530
-------- new ---------
9780375713750
-------- new ---------
9780099428381
-------- new ---------
9780743287968
-------- new ---------
9781583762059
-------- new ---------
9780471469339
-------- new ---------
9780471292517
-------- new ---------
9780393061635
-------- new ---------
9780449003886
-------- new ---------
9

9780060738259
-------- new ---------
9780312948962
-------- new ---------
9781593979263
-------- new ---------
9780747267621
-------- new ---------
9780330371247
-------- new ---------
9780061126529
-------- new ---------
9781857988376
-------- new ---------
9780553574012
-------- new ---------
9780803261839
-------- new ---------
9780345329561
-------- new ---------
9781400130214
-------- new ---------
9781400130221
-------- new ---------
9780345339300
-------- new ---------
9780345334244
-------- new ---------
9780345345110
-------- new ---------
9780345339935
-------- new ---------
9780345329554
-------- new ---------
9780143104889
-------- new ---------
9780345370112
-------- new ---------
9780517659571
-------- new ---------
9780809599813
-------- new ---------
9780751536829
-------- new ---------
9781569473641
-------- new ---------
9780743298025
-------- new ---------
9780345457745
-------- new ---------
9780785117681
-------- new ---------
9780785114932
-------- new ---------
9

9781568028347
-------- new ---------
9780810957909
-------- new ---------
9780810957893
-------- new ---------
9780810970571
-------- new ---------
9780060934729
-------- new ---------
9781590593899
-------- new ---------
9780596006624
-------- new ---------
9780553803709
-------- new ---------
9781596870413
-------- new ---------
9780553299496
-------- new ---------
9780553569971
-------- new ---------
9780140172133
-------- new ---------
9780553293425
-------- new ---------
9781857989342
-------- new ---------
9780553293432
-------- new ---------
9780449015414
-------- new ---------
9780060556525
-------- new ---------
9780380010325
-------- new ---------
9780380699179
-------- new ---------
9780142405741
-------- new ---------
9781598530018
-------- new ---------
9780786848935
-------- new ---------
9780521288842
-------- new ---------
9780192862099
-------- new ---------
9780673393555
-------- new ---------
9780316015844
-------- new ---------
9781593081256
-------- new ---------
9

-------- new ---------
9780300101621
-------- new ---------
9780393324945
-------- new ---------
9780321478061
-------- new ---------
9780060598969
-------- new ---------
9781859842843
-------- new ---------
9781859840542
-------- new ---------
9781859843833
-------- new ---------
9780860916284
-------- new ---------
9781859843987
-------- new ---------
9780452284982
-------- new ---------
9781859843406
-------- new ---------
9781859847862
-------- new ---------
9781883477370
-------- new ---------
9780141185163
-------- new ---------
9780809078677
-------- new ---------
9781590171103
-------- new ---------
9783462032178
-------- new ---------
9780345483171
-------- new ---------
9781556434242
-------- new ---------
9781932416237
-------- new ---------
9781932416121
-------- new ---------
9780241142462
-------- new ---------
9782290328248
-------- new ---------
9781590170700
-------- new ---------
9781932416381
-------- new ---------
9781932416480
-------- new ---------
9781932416015
-

9781581342475
-------- new ---------
9781581346084
-------- new ---------
9781400097036
-------- new ---------
9780449001172
-------- new ---------
9780345440839
-------- new ---------
9780761327059
-------- new ---------
9780142003565
-------- new ---------
9780140097368
-------- new ---------
9780852237519
-------- new ---------
9780786849604
-------- new ---------
9780786851492
-------- new ---------
9780439443364
-------- new ---------
9780786855032
-------- new ---------
9780786855018
-------- new ---------
9780307344670
-------- new ---------
9780862786021
-------- new ---------
9783548603209
-------- new ---------
9780307350053
-------- new ---------
9788497939201
-------- new ---------
9788484412472
-------- new ---------
9780307343109
-------- new ---------
9781400085910
-------- new ---------
9780739336502
-------- new ---------
9780375846724
-------- new ---------
9780375846731
-------- new ---------
9780807210475
-------- new ---------
9780375847226
-------- new ---------
9